<a href="https://colab.research.google.com/github/cij1012/Dacon_study/blob/main/%EC%B2%AD%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 가져오기

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import matplotlib.pyplot as plt

In [8]:
path = '/content/drive/MyDrive/Colab Notebooks/blue house'
train_data = pd.read_csv(path + '/train.csv').iloc[:,1:].dropna(how='any')
train_data.index = range(len(train_data))
test_data = pd.read_csv(path + '/test.csv').iloc[:,1:]
submission = pd.read_csv(path + '/sample_submission.csv')

In [9]:
train_data.shape, test_data.shape

((39992, 2), (5000, 1))

In [10]:
train_data.head()

,category,data
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [11]:
test_data.head()

,data
0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...
1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...
2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요
3,국민위원에가 삼성편만들어요. 삼성에서 11년간 일하고 혈암과 백혈병 진단을 받은 ...
4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부..."


# 데이터 전처리

In [12]:
def clean_txt(txt):
    corpus = []
    for i in range(0, len(txt)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(txt[i])) # (1,2,3) 3에있는 1을 2로 대체
        review = re.sub(r'\d+','',str(txt[i])) # 숫자 제거
        review = review.lower() # 소문자
        review = re.sub(r'\s+',' ',review) # 공백 제거
        review = re.sub(r'<[^>]+>','',review) # Html 태그 제거 
        review = re.sub(r'\s',' ',review) # Html 태그 제거후 다시 공백제거
        review = re.sub(r'^\s+','',review) # 첫칸 공백 제거  --- ^ 뒤의 문자로 문자열이 시작
        review = re.sub(r'\s+$','',review) # 마지막칸 공백 제거 --- $ 앞의 문자로 문자열이 종료
        corpus.append(review)
    return corpus
        

In [13]:
train_data.data = clean_txt(train_data.data)
test_data.data = clean_txt(test_data.data)

In [14]:
train_data.head()

,category,data
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,1,"티비 세,세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠시 딴일..."
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


cleansing string punctuation

In [15]:
"""
개행문자 제거
"""


train_data_text = list(train_data['data'])

train_clear_text = []

for i in tqdm(range(len(train_data_text))):
    train_clear_text.append(str(train_data_text[i]).replace('\\n',''))  
train_data['clear_text'] = train_clear_text
train_data.head()

100%|██████████| 39992/39992 [00:00<00:00, 374840.74it/s]


,category,data,clear_text
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올...","빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,1,"티비 세,세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠시 딴일...","티비 세,세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠시 딴일..."
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [16]:
"""
특수문자 제거
"""
train_clear_text = list(train_data['clear_text'])

train_clear_text2 = []

for text in train_clear_text:
    temp = re.sub('[-=+,#:;//●<>▲\?:^$.☆!★()Ⅰ@*\"※~>`\'…》]', ' ', text)
    train_clear_text2.append(temp)
train_data['clear_text'] = train_clear_text2
train_data.head()

,category,data,clear_text
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...,신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...,학교이름에 남자 도 붙여주세요 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올...",빙상연맹 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요 최근 동계올...
3,1,"티비 세,세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠시 딴일...",티비 세 세 관람가도 연령확인 의무화 하자 제기 에전에 티비를 보다가 잠시 딴일...
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...,무더운 여름철엔 남성들도 시원한 자율복장을 해야 무더운 여름철에는 남성들도 노넥타...


In [17]:
"""
test셋도 마찬가지로
"""
test_data_text = list(test_data['data'])

test_clear_text = []

for i in tqdm(range(len(test_data_text))):
  test_clear_text.append(test_data_text[i].replace('\\n', ' '))
test_data['clear_text'] = test_clear_text
test_data.head()

100%|██████████| 5000/5000 [00:00<00:00, 299289.58it/s]


,data,clear_text
0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이 법으로 인해 보호받지 못하...
1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에 *지역 어린이...
2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요
3,국민위원에가 삼성편만들어요. 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...,국민위원에가 삼성편만들어요. 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...
4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부...","방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부..."


In [18]:
test_clear_text = list(test_data['clear_text'])

test_clear_text2 = []

for text in test_clear_text:
  temp = re.sub('[-=+,#:;//●<>▲\?:^$.☆!★()Ⅰ@*\"※~>`\'…》]', ' ', text)
  test_clear_text2.append(temp)
test_data['clear_text'] = test_clear_text2
test_data.head()

,data,clear_text
0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...,소년법 폐지해주세요 법 아래에서 보호받아야 할 아이들이 법으로 인해 보호받지 못하...
1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...,국공립 유치원 증설에 관하여 국공립 유치원 부지 학보와건립및 증설에 지역 어린이...
2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요,나경원파면 나경원의원의 동계올림픽 위원을 파면해 주세요
3,국민위원에가 삼성편만들어요. 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...,국민위원에가 삼성편만들어요 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...
4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부...",방과후 유치원 어린이집 영어교육을 유지시켜주세요 저는 아이 셋 키우는 평범한 주부...


# Mecab을 이용하여 토큰화하기
- colab에서 mecab사용할 수 있도록 설치

In [19]:
# !pip install konlpy

In [24]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 33), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [25]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [26]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 67.3MB/s 
     |████████████████████████████████| 460kB 47.7MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-12-15 11:34:10--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22c2:513, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downlo

In [27]:
from konlpy.tag import Mecab
mecab = Mecab() # 윈도에서 Mecab 사용하기--https://hong-yp-ml-records.tistory.com/91

In [28]:
"""
mecab.nouns 테스트
"""

mecab.nouns(list(train_data['clear_text'])[9])

['수백만',
 '원',
 '녹',
 '취록',
 '증거',
 '인정',
 '사람',
 '죄',
 '실직',
 '평생',
 '꽃뱀',
 '성매매',
 '범',
 '저',
 '년',
 '개월',
 '간',
 '주민',
 '등록증',
 '변조',
 '신상',
 '위조',
 '피의자',
 '동안',
 '협박',
 '폭행',
 '만남',
 '거부',
 '직장',
 '전화',
 '실직',
 '명',
 '가늠',
 '수',
 '동호인',
 '꽃뱀',
 '성매매',
 '범',
 '허위',
 '사실',
 '동영상',
 '유포',
 '저',
 '꽃뱀',
 '성매매',
 '범',
 '폭행',
 '범',
 '차례',
 '형사',
 '고소',
 '검찰',
 '조사',
 '중',
 '건과',
 '항고',
 '중',
 '건',
 '번',
 '소건',
 '범죄',
 '중',
 '건',
 '모욕죄',
 '기소',
 '제',
 '때',
 '상습',
 '범죄자',
 '피의자',
 '현재',
 '저',
 '위협',
 '목적',
 '제',
 '소속',
 '클럽',
 '밴드',
 '꽃뱀',
 '운운',
 '댓글',
 '게시',
 '정치',
 '인',
 '사',
 '년',
 '모',
 '시장',
 '친구',
 '년',
 '시장',
 '후보',
 '선거',
 '본부',
 '장',
 '현재',
 '민주당',
 '위원장',
 '친분',
 '과시',
 '돈',
 '구장',
 '허위',
 '사실',
 '유포',
 '동호인',
 '증인',
 '매수',
 '친구',
 '동호인',
 '거짓',
 '참고인',
 '진술',
 '등',
 '범죄',
 '방조',
 '저',
 '생각',
 '치',
 '악몽',
 '피의자',
 '목소리',
 '수차례',
 '쉼',
 '눈물',
 '피의자',
 '범죄',
 '수백만',
 '원',
 '녹',
 '취록',
 '증거',
 '제출',
 '증거',
 '저',
 '꽃뱀',
 '성매매',
 '폭행',
 '범',
 '제',
 '제출',
 '녹',
 '취록',
 '자료',
 '증거',
 '인

불용어 처리

In [29]:
stop_df = pd.read_csv('/content/한국어불용어100.txt', sep = '\t', header = None, names = ['형태','품사','비율'])
stop_df.tail()

,형태,품사,비율
95,원,NNB,0.000492
96,잘,MAG,0.000491
97,통하,VV,0.000487
98,소리,NNG,0.000486
99,놓,VX,0.000483


In [30]:
stop_df.loc[100] = '가'
stop_df.loc[101] = '합니다'

In [31]:
stop_words = list(stop_df.형태)

outlier 삭제

In [32]:
ind_list = [24885, 14916, 14605, 6641, 17406, 26957, 2175, 6885, 8947, 14966, 8198, 25955, 39167,21707, 12678,
           3023, 31971, 3730, 37153, 33481, 33369, 12927, 30773, 36431,12373,  37525,27530, 8958, 16884, 18072,
           4478, 7940, 16400, 16656]
train_data = train_data.query('index not in @ind_list')
train_data.index = range(0, len(train_data))

get nouns using Mecab

In [33]:
X_train = []  # dict같은 형태로 저장(?)

text = list(train_data['clear_text'])

for i in tqdm(range(len(text))):
    temp = []
    temp = mecab.nouns(text[i])
    temp = [word for word in temp if not word in stop_words] # 불용어 제거
    temp = [word for word in temp if len(word) > 1]
    X_train.append(temp)

100%|██████████| 39958/39958 [01:12<00:00, 549.22it/s]


In [34]:
X_test = []

text = list(test_data['clear_text'])

for i in tqdm(range(len(text))):
    temp = []
    temp = mecab.nouns(text[i])
    temp = [word for word in temp if not word in stop_words] # 불용어 제거
    temp = [word for word in temp if len(word) > 1]
    X_test.append(temp)

100%|██████████| 5000/5000 [00:09<00:00, 547.12it/s]


# 모델링 - Tokenizer

In [35]:
import tensorflow as tf

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop

In [37]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [40]:
threshold = 11  # 역치- 이것보다 많이 나온 단어는 유의미하다...-> 파라미터의 하나가 될 수 있을 듯
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어 개수 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

"""
단어 & 빈도수의 쌍을 key 와 value로 받는다.
"""
for key, value in tokenizer.word_counts.items():  ## tokenizer.word_counts -> key 단어 value 빈도수로 구성된 dict
  total_freq = total_freq + value

  if(value < threshold):
    rare_cnt = rare_cnt + 1
    rare_freq = rare_freq + value

print('단어 집합의 크기: ', total_cnt)
print('등장 빈도수가 %s번 이하인 희귀 단어의 수 : %s'%(threshold-1, rare_cnt))
print('단어 집합에서 희귀 단어의 비율 : ',(rare_cnt/total_cnt))
print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : ',(rare_freq/total_freq),"%")

단어 집합의 크기:  45144
등장 빈도수가 10번 이하인 희귀 단어의 수 : 32528
단어 집합에서 희귀 단어의 비율 :  0.7205387205387206
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 :  0.032528203776260645 %


In [41]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 : ', vocab_size)

단어 집합의 크기 :  12617
